# Гипотеза

Добавление в окрестность не 4 а 6 точек вокруг и удаление из обучения ритмограммы № 107, 108, 109, 127 даст лучший скор.  
Пока лучшее `mean test_score = 0.7161425084850291`

> **итог**  
> при добавлении пятой и шестой точки в окрестность дало улучшение сокора `mean test_score = 0.7718847140739808`  
> при дополнительном удалении некорректных ритмограмм скор снизился `mean test_score = 0.7682827383694755`

In [6]:
import pandas as pd
import pickle
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import f1_score
from scipy.stats import entropy

In [3]:
# Const
RANDOM_STATE = 0
min_rr = 460
max_rr = 963*1.2

print(min_rr, '/', max_rr)

460 / 1155.6


# Полезные методы

In [15]:
def make_XY(df):
    """
    Подготовка признаков для обучения модели
    df: dataset
    return:
    X: numpy array with features, values are scaled
    y: numpy array of target labels
    """
    X = df.drop(['y','id','time'], axis=1)
    y = df.y.to_numpy()

    scaler = StandardScaler().fit(X.to_numpy())
    X = scaler.transform(X.to_numpy())

    with open('scaler_hypothesis_8.pkl', 'wb') as f:
        pickle.dump(scaler, f)
        
    print('scaler_hypothesis_8.pkl was saved in output directory')

    return X, y


def get_train_test_indexes(X,y):
    """
    X,y: numpy arrays with features and target
    return stratified indexes:
        train_indexes: indexes for train data
        test_indexes: indexes for test data
    """
    skf = StratifiedKFold()
    folds = dict()
    for i, (train_indexes, test_indexes) in enumerate(skf.split(X,y)):
        folds[i] = {
            'train_indexes': train_indexes,
            'test_indexes': test_indexes
        }
    return folds[0]['train_indexes'].tolist(), folds[0]['test_indexes'].tolist()


def plot_RR(rr_ids, data):
    """
    Рисует графики ритмограмм с разметкой аномальных участков
    rr_ids: список идентификаторов ритмограмм
    data: набор данных
    """
    df = data.set_index('time').copy()
    for rr_id in rr_ids:
        fig = plt.figure(figsize=(20,5))
        plt.title(f'R-R №{rr_id}')
        plt.plot(df[df.id == rr_id].x, '-o', zorder=1)
        df_anomaly = df[(df.id == rr_id)&(df.y == 1)].reset_index()
        df_anomaly['time-diff'] = df_anomaly.time.diff()
        split_indexes = df_anomaly[df_anomaly['time-diff'] > 1000].index
        split_indexes = list(split_indexes)
        split_indexes.append(0)
        split_indexes.sort()
        len_spl = len(split_indexes)
        for i in range(len_spl):
            if i == len_spl-1:
                mask = (df_anomaly.index >= split_indexes[i])
                plt.plot(df_anomaly[mask].time, df_anomaly[mask].x, '-o',
                     label='аномальный участок', color='red', zorder=2)
            else:
                mask = (df_anomaly.index >= split_indexes[i])&(df_anomaly.index < split_indexes[i+1])
                plt.plot(df_anomaly[mask].time, df_anomaly[mask].x, '-o',
                         color='red', zorder=2)
        plt.legend()
        plt.xlabel('R-R timeline, ms')
        plt.ylabel('R-R interval')
        plt.show()
        
        
def entropy1(labels, base=None):
    _, counts = np.unique(labels, return_counts=True)
    return entropy(counts, base=base)


def make_dataset(data):
    """
    Автоматизация подготовки датасета
    """
    df = data.copy()
    ids = df.id.unique()
    for rr_id in ids:
        mask = (df.id==rr_id)
        df.loc[mask, 'entropy'] = df[mask].x.rolling(20).apply(entropy1).fillna(method='bfill')
        df.loc[mask, 'x_diff'] = df[mask].x.diff()
        df.fillna(method='bfill', inplace=True)
        df.loc[mask, 'x_deviation_median'] = df[mask].x.median() - df[mask].x.values
        for i in range(1,7):
            # добавим в признаки 6 следующих точки
            df.loc[mask, f'x+{i}'] = df[mask].x.shift(-i)
            # и 6 предыдущие точки
            df.loc[mask, f'x-{i}'] = df[mask].x.shift(i)
        df.loc[mask, 'x-(x+1)'] = df[mask].x.values - df.loc[mask, 'x+1'].values
        df.loc[mask, 'x-(x+2)'] = df[mask].x.values - df.loc[mask, 'x+2'].values
        df.loc[mask, 'x-(x-2)'] = df[mask].x.values - df.loc[mask, 'x-2'].values
        df.loc[mask, 'x-(x+3)'] = df[mask].x.values - df.loc[mask, 'x+3'].values
        df.loc[mask, 'x-(x-3)'] = df[mask].x.values - df.loc[mask, 'x-3'].values
        df.loc[mask, 'x-(x+4)'] = df[mask].x.values - df.loc[mask, 'x+4'].values
        df.loc[mask, 'x-(x-4)'] = df[mask].x.values - df.loc[mask, 'x-4'].values
        df.loc[mask, 'x-(x+5)'] = df[mask].x.values - df.loc[mask, 'x+5'].values
        df.loc[mask, 'x-(x-5)'] = df[mask].x.values - df.loc[mask, 'x-5'].values
        df.loc[mask, 'x-(x+6)'] = df[mask].x.values - df.loc[mask, 'x+6'].values
        df.loc[mask, 'x-(x-6)'] = df[mask].x.values - df.loc[mask, 'x-6'].values
        df.fillna(method='bfill', inplace=True)
        df.fillna(method='ffill', inplace=True)
        
    return df

# Данные

In [7]:
data = pd.read_csv('../input/cardiospikecompetition/train.csv')
df = make_dataset(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60487 entries, 0 to 60486
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  60487 non-null  int64  
 1   time                60487 non-null  int64  
 2   x                   60487 non-null  int64  
 3   y                   60487 non-null  int64  
 4   entropy             60487 non-null  float64
 5   x_diff              60487 non-null  float64
 6   x_deviation_median  60487 non-null  float64
 7   x+1                 60487 non-null  float64
 8   x-1                 60487 non-null  float64
 9   x+2                 60487 non-null  float64
 10  x-2                 60487 non-null  float64
 11  x+3                 60487 non-null  float64
 12  x-3                 60487 non-null  float64
 13  x+4                 60487 non-null  float64
 14  x-4                 60487 non-null  float64
 15  x+5                 60487 non-null  float64
 16  x-5 

# Замена выбросов

In [8]:
print(f'количество выбрасов выше {max_rr} =', len(df[df.x > max_rr]))
print(f'количество выбрасов ниже {min_rr} =', len(df[df.x < min_rr]))

количество выбрасов выше 1155.6 = 1015
количество выбрасов ниже 460 = 10220


In [9]:
out_cnt = len(df[(df.x > max_rr)|(df.x < min_rr)].y)
df.loc[df.x > max_rr, 'x'] = pd.NA
df.loc[df.x < min_rr, 'x'] = pd.NA

print('количество NA после мьютирования =', len(df[df.x.isna()]))
if len(df[df.x.isna()]) == out_cnt:
    print('выбросы замьютированы корректно')
else:
    print('мьютирование выбросов прошло некорректно...')

количество NA после мьютирования = 11235
выбросы замьютированы корректно


In [10]:
# индексы, которые были выялены как выбросы
out_ind = df[df.x.isna()].index

In [11]:
# test
df.x.ffill(inplace=True)
# проверка на корректность мьютирования
ind = np.random.randint(len(out_ind))
display(df.loc[[out_ind[ind]-1, out_ind[ind]], 'x'])
if df.loc[out_ind[ind]-1, 'x'] == df.loc[out_ind[ind], 'x']:
    print('\nПроверка прошла успешно! :)')
else:
    print('\nДанные в соседних точках не совпадают... :(')

20215    480
20216    480
Name: x, dtype: int64


Проверка прошла успешно! :)


In [13]:
# сохраним датасет в с текущими признаками, предварительно удалив строки с пропусками
with open('dataset_hypothesis_8.pkl', 'wb') as f:
    pickle.dump(df, f)
    
df.to_csv('dataset_hypothesis_8.csv')

# X, y

In [16]:
# make X and y
X,y = make_XY(df)

scaler_hypothesis_8.pkl was saved in output directory


# Cross validation

## ver 1

In [17]:
model = lgb.LGBMClassifier(n_estimators=3860, learning_rate=0.01, random_state=RANDOM_STATE, n_jobs=-1)

cv_result = cross_validate(model, X, y, cv=StratifiedKFold(), scoring='f1')

print('test_score:', cv_result['test_score'])
print('mean test_score =', cv_result['test_score'].mean())
print('current best mean test_score = 0.7161425084850291')

test_score: [0.84874204 0.76762262 0.63091957 0.82703038 0.78510896]
mean test_score = 0.7718847140739808
current best mean test_score = 0.7161425084850291


>скор улучшился!

## ver 2

Удалим некорректные ритмограммы: 107, 108, 109, 127

In [18]:
# количество удаляемых точек
len(df[(df.id==107)|(df.id==108)|(df.id==109)|(df.id==127)])

5087

In [20]:
# test
del_cnt = len(df[(df.id==107)|(df.id==108)|(df.id==109)|(df.id==127)])
df_cut_len = len(df.drop(df[(df.id==107)|(df.id==108)|(df.id==109)|(df.id==127)].index))
if df_cut_len == len(df)-del_cnt:
    print('Ок, всё корректно, можно отрезать :)')
else:
    print('Что-то не так...')

Ок, всё корректно, можно отрезать :)


In [21]:
if df_cut_len == len(df)-del_cnt:
    print(f'Было {len(df)}')
    df.drop(df[(df.id==107)|(df.id==108)|(df.id==109)|(df.id==127)].index, inplace=True)
    print(f'Стало {len(df)}')

Было 60487
Стало 55400


In [22]:
X_v2, y_v2 = make_XY(df)

scaler_hypothesis_8.pkl was saved in output directory


In [23]:
model_v2 = lgb.LGBMClassifier(n_estimators=3860, learning_rate=0.01, random_state=RANDOM_STATE, n_jobs=-1)

cv_result = cross_validate(model_v2, X_v2, y_v2, cv=StratifiedKFold(), scoring='f1')

print('test_score:', cv_result['test_score'])
print('mean test_score =', cv_result['test_score'].mean())
print('current best mean test_score = 0.7718847140739808')

test_score: [0.85705388 0.64327485 0.72302273 0.8424015  0.77566073]
mean test_score = 0.7682827383694755
current best mean test_score = 0.7718847140739808
